In [1]:
import pandas as pd
from fuzzywuzzy import process

# Load the datasets
file_1_path = 'updated_beta_cause_data.csv'  # Update with the actual path to your first file
file_2_path = 'clean-environmental-dataset.csv'  # Update with the actual path to your second file

file_1 = pd.read_csv(file_1_path)
file_2 = pd.read_csv(file_2_path)

# Assuming 'Company Name' is the column in both files to match on
# Update 'Company Name' to the actual column names in your files if different
company_names_file_1 = file_1['Company Name'].tolist()
company_names_file_2 = file_2['Company'].tolist()

# Function to apply fuzzy matching and find the best match for each company name in file_1 from file_2
def get_best_match(name, list_names, threshold=80):
    best_match = process.extractOne(name, list_names, score_cutoff=threshold)
    return best_match[0] if best_match else None

# Apply fuzzy matching to find the best match for each company name
file_1['Matched Company Name'] = file_1['Company Name'].apply(lambda x: get_best_match(x, company_names_file_2))

# Merge the two files based on the matched company names
merged_data = pd.merge(file_1, file_2, left_on='Matched Company Name', right_on='Company Name', how='left')

# Save the merged data to a new CSV file
merged_file_path = 'final_dataset.csv'  # Specify your desired output file name
merged_data.to_csv(merged_file_path, index=False)

print(f"Merged data saved to {merged_file_path}.")


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Load the merged data
merged_file_path = 'final_dataset.csv'  # Update with the actual path to your merged file
data = pd.read_csv(merged_file_path)
# Assuming there are duplicate 'Country' columns named 'Country_x' and 'Country_y'
# We'll keep 'Country_x' and drop 'Country_y'
# Update column names as per your actual data
data = data.drop(columns=['Country_y'])

# Assuming 'Industry' column contains comma-separated values for industries
# First, we'll split the 'Industry' column into a list of industries
data['Industry'] = data['Industry'].apply(lambda x: x.split(', ') if pd.notnull(x) else [])

# Now, we'll use pandas' get_dummies to perform multi-hot encoding
# This creates a new dataframe with binary columns for each industry
industry_encoded = data['Industry'].str.join('|').str.get_dummies()

# Merge the encoded industries back into the original dataframe
data_encoded = pd.concat([data, industry_encoded], axis=1)

# Optionally, drop the original 'Industry' column if it's no longer needed
data_encoded = data_encoded.drop(columns=['Industry'])

# Save the updated data to a new CSV file
updated_file_path = 'updated_merged_file.csv'  # Specify your desired output file name
data_encoded.to_csv(updated_file_path, index=False)

print(f"Updated data saved to {updated_file_path}.")